# Http Response

## Usage

To construct a response packet you have a variety of facilities available.

Previously we saw how to parse HTTP responses using `HttpParser`.  Of-course, we can also construct a response packet using `HttpParser` class.

In [2]:
from proxy.http.parser import HttpParser, httpParserTypes
from proxy.common.constants import HTTP_1_1

response = HttpParser(httpParserTypes.RESPONSE_PARSER)
response.code = b'200'
response.reason = b'OK'
response.version = HTTP_1_1

print(response.build_response())

b'HTTP/1.1 200 OK\r\nContent-Length: 0\r\n\r\n'


But, this is a painful way to construct responses.  Hence, other high level abstractions are available.

Example, following one liner will give us the same response packet.

In [7]:
from proxy.http.responses import okResponse

print(okResponse().tobytes())

b'HTTP/1.1 200 OK\r\nContent-Length: 0\r\n\r\n'


Notice how `okResponse` will always add a `Content-Length` header for you.

You can also customize other headers

In [5]:
response = okResponse(
    headers={
        b'X-Custom-Header': b'my value',
    },
)

print(response.tobytes())

b'HTTP/1.1 200 OK\r\nX-Custom-Header: my value\r\nContent-Length: 0\r\n\r\n'


Let's add some content to our response packet

In [6]:
response = okResponse(
    content=b'Hello World',
    headers={
        b'X-Custom-Header': b'my value',
    },
)

print(response.tobytes())

b'HTTP/1.1 200 OK\r\nX-Custom-Header: my value\r\nContent-Length: 11\r\n\r\nHello World'


Note, how `okResponse` automatically added a `Content-Length` header for us.

Depending upon `--min-compression-length` flag, `okResponse` will also perform compression for content.

Example, default value for min compression length is 20.

In [16]:
response = okResponse(
    content=b'H' * 21,
    headers={
        b'X-Custom-Header': b'my value',
    },
)

print(response.tobytes())

b'HTTP/1.1 200 OK\r\nX-Custom-Header: my value\r\nContent-Encoding: gzip\r\nContent-Length: 23\r\n\r\n\x1f\x8b\x08\x00F\x0e\xf1a\x02\xff\xf3\xf0\xc0\x02\x00h\x81?s\x15\x00\x00\x00'


You can pass a custom value for `min_compression_length` kwarg to `okResponse`.

In [6]:
response = okResponse(
    content=b'H' * 21,
    headers={
        b'Host': b'jaxl.com',
    },
    min_compression_length=21,
)

print(response.tobytes())

b'HTTP/1.1 200 OK\r\nHost: jaxl.com\r\nContent-Length: 21\r\n\r\nHHHHHHHHHHHHHHHHHHHHH'


Internally, `okResponse` uses `build_http_response` and hence you can also pass any argument also accepted by `build_http_response`.  Example, it supports a `conn_close` argument which will add a `Connection: close` header.  Simply, pass `conn_close=True`.

In [7]:
response = okResponse(
    content=b'Hello World',
    headers={
        b'Host': b'jaxl.com',
    },
    conn_close=True,
)

print(response.tobytes())

b'HTTP/1.1 200 OK\r\nHost: jaxl.com\r\nContent-Length: 11\r\nConnection: close\r\n\r\nHello World'


## Chunked Encoding

You can also send chunked encoded responses.

In [12]:
from proxy.http.parser import ChunkParser

chunks = ChunkParser.to_chunks(b'Hello World', chunk_size=5)
response = okResponse(
    content=chunks,
    headers={
        b'Transfer-Encoding': b'chunked',
    },
    # Avoid compressing chunks for demo purposes here
    # Ideally you should omit this flag and send
    # compressed chunks.
    min_compression_length=len(chunks),
)

print(response.tobytes())

b'HTTP/1.1 200 OK\r\nTransfer-Encoding: chunked\r\n\r\n5\r\nHello\r\n5\r\n Worl\r\n1\r\nd\r\n0\r\n\r\n'


If we omit the `min_compression_length` flag

In [13]:
response = okResponse(
    content=chunks,
    headers={
        b'Transfer-Encoding': b'chunked',
    },
)

print(response.tobytes())

b'HTTP/1.1 200 OK\r\nTransfer-Encoding: chunked\r\nContent-Encoding: gzip\r\n\r\n\x1f\x8b\x08\x00\xd3\n\xf1a\x02\xff3\xe5\xe5\xf2H\xcd\xc9\xc9\xe7\xe52\xe5\xe5R\x08\xcf/\xca\xe1\xe52\xe4\xe5J\xe1\xe52\xe0\xe5\xe2\xe5\x02\x00\x90S\xbb/\x1f\x00\x00\x00'
